In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Lager inputs der brukeren kan taste inn verdiene sine
def Verdier():
    print("Lånebeløp")
    låne_sum = float(input())
    print("Nedbetalingsår")
    låne_periode = float(input())
    lån_termin = int(12*låne_periode)
    print("Rente")
    lån_rente = float(input())
    return låne_sum, låne_periode, lån_termin, lån_rente

def Kalk(låne_sum, låne_periode, lån_termin, lån_rente):   
    R = 1 + (lån_rente/100)/12
    X = låne_sum*(R**lån_termin)*(1-R)/(1-R**lån_termin) # Funksjon for annuitetslån
    Månedlig_rente = []
    Månedlig_saldo = []
    
    for i in range(1, lån_termin+1):
        Rente = låne_sum*(R-1)
        låne_sum = låne_sum-(X-Rente)
        Månedlig_rente = np.append(Månedlig_rente,Rente)
        Månedlig_saldo = np.append(Månedlig_saldo, låne_sum)
    # Skriver ut hva beløpene blir
    print("Den årlige renten på dette lånet er: = " +str(lån_rente)+str(' %'))
    print("Lengden på lånet i måneder er: = " + str(lån_termin)+str(' måneder'))
    print("Månedlige betalinger for dette lånet blir på: = " +str(np.round(X,2)) + str(' Kr')) # Bruker round til å avrunde til to desimaler
    print("Den totale summen av renter betalt blir: = " + str(np.round(np.sum(Månedlig_rente),2)))

    # Prøver å plotte terminbeløp og rente i graf
    
    plt.plot(range(1, lån_termin+1), Månedlig_rente, lw=2)
    plt.axhline(y=X, color='red')
    plt.xlabel('Måned')
    plt.ylabel('Månedlig rentebeløp (Kr)')
    plt.show()   
    return X, R
        
def Terminbeløp_tabell (X, lån_termin, R, låne_sum, lån_rente):
    overskrift = '{0:10}{1:10}{2:10}{3:10}{4:20s}'
    verdier = '{0:5d}{1:12,.2f}{2:10,.2f}{3:12,.2f}{4:12,.2f}'
    print(overskrift.format('Betaling', 'Beløp', 'Rente', 'Avdrag', 'Restlån'))
    Restlån = abs(låne_sum)
    for Betaling in range(1, int(lån_termin)+1):
        Renter = abs(Restlån) * abs(lån_rente* .1/lån_termin)
        Avdrag = abs(X) - abs(Renter)
        Restlån -= abs(Avdrag)
        print(verdier.format(Betaling, X, Renter, Avdrag, Restlån))
        
def Regn(): 
    låne_sum, låne_periode, lån_termin, lån_rente = Verdier()
    X, R = Kalk(låne_sum, låne_periode, lån_termin, lån_rente)
    Terminbeløp_tabell(X, lån_termin, R, låne_sum, lån_rente)

Regn()
  
      

Lånebeløp
